## DX799S O1 Data Science Capstone (Summer 1 2025): ACTIVITY 1.2 ##

Each week, you will apply the concepts of that week to your Integrated Capstone Project’s dataset. In preparation for Milestone One, create a Jupyter Notebook (similar to in Module B, semester two) that illustrates these lessons. There are no specific questions to answer in your Jupyter Notebook files in this course; your general goal is to analyze your data, using the methods you have learned about in this course and in this program, and draw interesting conclusions. 

For Week 1, include concepts such as linear regression with polynomial terms, interaction terms, multicollinearity, variance inflation factor and regression, and categorical and continuous features. Complete your Jupyter Notebook homework by 11:59 pm ET on Sunday. 

In Week 7, you will compile your findings from your Jupyter Notebook homework into your Milestone One assignment for grading. For full instructions and the rubric for Milestone One, refer to the following link. 

In [89]:
# ===================================
# Useful Imports: Add more as needed
# ===================================

# Standard Libraries
import os
import time
import math
import io
import zipfile
import requests
from urllib.parse import urlparse
from itertools import chain, combinations

# Data Science Libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker  # Optional: Format y-axis labels as dollars
import seaborn as sns

# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    GridSearchCV, 
    RandomizedSearchCV, 
    RepeatedKFold
)
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector, f_regression, SelectKBest
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score



The following dataset, "Video Review", is a collection of information that was created based on reviewable video evidence that outlines the events that resulted in a concussion during punt players in the NFL 2016-2017 season. The target, Primary_Impact_Type, outlines if the concussion occurred from the impact of Helmet-to-Helmet, Helmet-to-Body, or Helmet-to-Ground.

In [90]:
#Video Review Dataset 

df_videoreview = pd.read_csv("video_review.csv")

label_encoder = LabelEncoder()

print("Object columns before encoding:")
print(df_videoreview.select_dtypes(include=['object']).columns)

for col in df_videoreview.select_dtypes(include=['object']).columns:
    df_videoreview[col] = label_encoder.fit_transform(df_videoreview[col].astype(str))


target_column = 'Primary_Impact_Type'  
X = df_videoreview.drop(columns=[target_column])
y = df_videoreview[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



Object columns before encoding:
Index(['Player_Activity_Derived', 'Turnover_Related', 'Primary_Impact_Type',
       'Primary_Partner_GSISID', 'Primary_Partner_Activity_Derived',
       'Friendly_Fire'],
      dtype='object')


In [91]:
#Video Review linear regression with categorical and continuous features

model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred_train = model.predict(X_train_scaled)

from sklearn.metrics import mean_squared_error, r2_score
print("MSE Video Review:", mean_squared_error(y_train, y_pred_train))
print("R² Video Review:", r2_score(y_train, y_pred_train))


MSE Video Review: 0.7820389432809272
R² Video Review: 0.2676004996667486


In [92]:
#Video Review linear regression with polynomial terms

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

model = LinearRegression()
model.fit(X_train_poly, y_train)

y_train_pred = model.predict(X_train_poly)

mse = mean_squared_error(y_train, y_train_pred)
r2 = r2_score(y_train, y_train_pred)

print("Mean Squared Error (MSE) of Video Review with Polynomial Terms:", mse)
print("R² Score of Video Review with Polynomial Terms:", r2)

Mean Squared Error (MSE) of Video Review with Polynomial Terms: 1.0023973916342858e-29
R² Score of Video Review with Polynomial Terms: 1.0


In [93]:
#Video Review linear regression with interaction terms

interaction = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
X_train_interact = interaction.fit_transform(X_train_scaled)

model = LinearRegression()
model.fit(X_train_interact, y_train)

y_train_pred = model.predict(X_train_interact)

mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

print("Mean Squared Error (MSE) Video Review with interaction terms:", mse_train)
print("R² Score Video Review with interaction terms:", r2_train)


Mean Squared Error (MSE) Video Review with interaction terms: 1.520427386937825e-29
R² Score Video Review with interaction terms: 1.0


In [94]:
#Video Review linear regression with multicollinearity

model = LinearRegression()
model.fit(X_train_scaled, y_train)
y_pred_train = model.predict(X_train_scaled)

mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

print("MSE Video Review with multicollinearity:", mse_train)
print("R² Video Review with multicollinearity:", r2_train)

coef_df = pd.DataFrame({'Feature': X.columns, 'Coefficient': model.coef_})
print("Coefficients:")
print(coef_df.sort_values(by='Coefficient', key=abs, ascending=False))


MSE Video Review with multicollinearity: 0.7820389432809272
R² Video Review with multicollinearity: 0.2676004996667486
Coefficients:
                            Feature   Coefficient
1                           GameKey  5.017196e-01
0                       Season_Year -4.723777e-01
4           Player_Activity_Derived -4.587435e-01
8                     Friendly_Fire -4.444616e-01
7  Primary_Partner_Activity_Derived  3.349750e-01
3                            GSISID  2.988874e-01
6            Primary_Partner_GSISID  1.899683e-01
2                            PlayID -7.620664e-02
5                  Turnover_Related  2.428613e-17


In [95]:
!pip install statsmodels


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [96]:
#Video Review linear regression with variance inflation factor and regression
from statsmodels.stats.outliers_influence import variance_inflation_factor

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)

def calculate_vif(X_df):
    vif_data = pd.DataFrame()
    vif_data["feature"] = X_df.columns
    vif_data["VIF"] = [variance_inflation_factor(X_df.values, i) for i in range(X_df.shape[1])]
    return vif_data

vif_df = calculate_vif(X_train_scaled_df)
print("Initial VIF values:")
print(vif_df.sort_values(by="VIF", ascending=False))

high_vif_features = vif_df[vif_df["VIF"] > 10]["feature"].tolist()
X_filtered = X_train_scaled_df.drop(columns=high_vif_features)


model = LinearRegression()
model.fit(X_filtered, y_train)
y_pred_train = model.predict(X_filtered)

mse = mean_squared_error(y_train, y_pred_train)
r2 = r2_score(y_train, y_pred_train)

print("MSE of Video Review with VIF:", mse)
print("R² Video Review with VIF:", r2)

coef_df = pd.DataFrame({'Feature': X_filtered.columns, 'Coefficient': model.coef_})
print("\nCoefficients (Reduced Model):")
print(coef_df.sort_values(by='Coefficient', key=abs, ascending=False))


Initial VIF values:
                            feature       VIF
0                       Season_Year  4.689585
1                           GameKey  3.588601
8                     Friendly_Fire  2.387368
3                            GSISID  2.289465
7  Primary_Partner_Activity_Derived  2.199185
2                            PlayID  1.422762
6            Primary_Partner_GSISID  1.415988
4           Player_Activity_Derived  1.304255
5                  Turnover_Related       NaN
MSE of Video Review with VIF: 0.7820389432809272
R² Video Review with VIF: 0.2676004996667486

Coefficients (Reduced Model):
                            Feature   Coefficient
1                           GameKey  5.017196e-01
0                       Season_Year -4.723777e-01
4           Player_Activity_Derived -4.587435e-01
8                     Friendly_Fire -4.444616e-01
7  Primary_Partner_Activity_Derived  3.349750e-01
3                            GSISID  2.988874e-01
6            Primary_Partner_GSISID  1.899683

/usr/local/python/3.12.1/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


The next dataset, Injury Record, looks to determine the relationship between the playing surface and the injury and performance of NFL athletes. The Injury Record dataset accounts for 105 lower-limbs injuries that occurred over two seasons during the regular NFL season and provides information on the surface the game occurred on and the number of days the player missed due to injury (or how severe it was). The target in this case is surface which lists the type of surface (synethic or natural) the field was when the injury occurred.

In [97]:
df_injuryrecord = pd.read_csv("InjuryRecord.csv")

In [98]:
label_encoder = LabelEncoder()

print("Object columns before encoding:")
print(df_injuryrecord.select_dtypes(include=['object']).columns)

for col in df_injuryrecord.select_dtypes(include=['object']).columns:
    df_injuryrecord[col] = label_encoder.fit_transform(df_injuryrecord[col].astype(str))


target_column = 'Surface'  
X_injury = df_injuryrecord.drop(columns=[target_column])
y_injury = df_injuryrecord[target_column]

X_train, X_test, y_train, y_test = train_test_split(X_injury, y_injury, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Object columns before encoding:
Index(['GameID', 'PlayKey', 'BodyPart', 'Surface'], dtype='object')


In [99]:
#Injury Record linear regression with categorical and continuous features

model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred_train = model.predict(X_train_scaled)

from sklearn.metrics import mean_squared_error, r2_score
print("MSE Injury Record:", mean_squared_error(y_train, y_pred_train))
print("R² Injury Record:", r2_score(y_train, y_pred_train))


MSE Injury Record: 0.23365898721080608
R² Injury Record: 0.06057104629091281


In [100]:
#Injury Record linear regression with polynomial terms
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

model = LinearRegression()
model.fit(X_train_poly, y_train)

y_train_pred = model.predict(X_train_poly)

mse = mean_squared_error(y_train, y_train_pred)
r2 = r2_score(y_train, y_train_pred)

print("Mean Squared Error (MSE) of Injury Record with Polynomial Terms:", mse)
print("R² Score of Injury Record with Polynomial Terms:", r2)

Mean Squared Error (MSE) of Injury Record with Polynomial Terms: 0.17576404897108824
R² Score of Injury Record with Polynomial Terms: 0.29333838772649645


In [101]:
#Injury Record linear regression with interaction terms

interaction = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
X_train_interact = interaction.fit_transform(X_train_scaled)

model = LinearRegression()
model.fit(X_train_interact, y_train)

y_train_pred = model.predict(X_train_interact)

mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

print("Mean Squared Error (MSE) Injury Record with interaction terms:", mse_train)
print("R² Score Injury Record with interaction terms:", r2_train)


Mean Squared Error (MSE) Injury Record with interaction terms: 0.19614996908772048
R² Score Injury Record with interaction terms: 0.21137653453962624


In [102]:
#Injury Record linear regression with multicollinearity

model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred_train = model.predict(X_train_scaled)

mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

print("MSE Injury Record with multicollinearity:", mse_train)
print("R² Injury Record with multicollinearity:", r2_train)

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)

coef_df = pd.DataFrame({
    'Feature': X_train_scaled_df.columns,
    'Coefficient': model.coef_
})

print("Coefficients:")
print(coef_df.sort_values(by='Coefficient', key=abs, ascending=False))


MSE Injury Record with multicollinearity: 0.23365898721080608
R² Injury Record with multicollinearity: 0.06057104629091281
Coefficients:
     Feature   Coefficient
0  PlayerKey  2.895148e-01
1     GameID -2.254470e-01
6     DM_M28  1.380293e-01
7     DM_M42 -9.425601e-02
2    PlayKey  5.508942e-02
3   BodyPart  2.208297e-03
5      DM_M7 -3.225481e-04
4      DM_M1  1.318390e-16


In [103]:
#Injury Record linear regression with variance inflation factor and regression
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)

def calculate_vif(X_df):
    vif_data = pd.DataFrame()
    vif_data["feature"] = X_df.columns
    vif_data["VIF"] = [variance_inflation_factor(X_df.values, i) for i in range(X_df.shape[1])]
    return vif_data

vif_df = calculate_vif(X_train_scaled_df)
print("Initial VIF values:")
print(vif_df.sort_values(by="VIF", ascending=False))

high_vif_features = vif_df[vif_df["VIF"] > 10]["feature"].tolist()
X_filtered = X_train_scaled_df.drop(columns=high_vif_features)


model = LinearRegression()
model.fit(X_filtered, y_train)
y_pred_train = model.predict(X_filtered)

mse = mean_squared_error(y_train, y_pred_train)
r2 = r2_score(y_train, y_pred_train)

print("MSE of Injury Record with VIF:", mse)
print("R² Injury Record with VIF:", r2)

coef_df = pd.DataFrame({'Feature': X_filtered.columns, 'Coefficient': model.coef_})
print("\nCoefficients (Reduced Model):")
print(coef_df.sort_values(by='Coefficient', key=abs, ascending=False))

Initial VIF values:
     feature        VIF
1     GameID  19.718543
0  PlayerKey  17.941774
6     DM_M28   3.368880
7     DM_M42   3.080271
2    PlayKey   1.553218
5      DM_M7   1.431353
3   BodyPart   1.063383
4      DM_M1        NaN
MSE of Injury Record with VIF: 0.24156478051584446
R² Injury Record with VIF: 0.02878570295168159

Coefficients (Reduced Model):
    Feature   Coefficient
4    DM_M28  1.147376e-01
5    DM_M42 -7.608983e-02
0   PlayKey  6.504045e-02
1  BodyPart -2.066456e-03
3     DM_M7  1.101684e-03
2     DM_M1 -2.151057e-16


/usr/local/python/3.12.1/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


The last dataset, PlayList, "contains information about each player-play in the dataset, to include the player’s assigned roster position, stadium type, field type, weather, play type, position for the play, and position group". This dataset was provided with the injury_record dataset so it provides additional information regarding the environment when a player's lower body injury occurred during these two NFL seasons. The target in this case will be "PlayType" which can include kickoff, run, pass, etc.

In [104]:
df_playlist = pd.read_csv("PlayList.csv")

label_encoder = LabelEncoder()

print("Object columns before encoding:")
print(df_playlist.select_dtypes(include=['object']).columns)

for col in df_playlist.select_dtypes(include=['object']).columns:
    df_playlist[col] = label_encoder.fit_transform(df_playlist[col].astype(str))


target_column = 'PlayType'  
X_playlist = df_playlist.drop(columns=[target_column])
y_playlist = df_playlist[target_column]

X_train, X_test, y_train, y_test = train_test_split(X_playlist, y_playlist, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Object columns before encoding:
Index(['GameID', 'PlayKey', 'RosterPosition', 'StadiumType', 'FieldType',
       'Weather', 'PlayType', 'Position', 'PositionGroup'],
      dtype='object')


In [105]:
#Playlist linear regression with categorical and continuous features
model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred_train = model.predict(X_train_scaled)

from sklearn.metrics import mean_squared_error, r2_score
print("MSE Playlist:", mean_squared_error(y_train, y_pred_train))
print("R² Playlist:", r2_score(y_train, y_pred_train))

MSE Playlist: 5.505702647235538
R² Playlist: 0.006451537032031829


In [106]:
#Playlist linear regression with polynomial terms
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

model = LinearRegression()
model.fit(X_train_poly, y_train)

y_train_pred = model.predict(X_train_poly)

mse = mean_squared_error(y_train, y_train_pred)
r2 = r2_score(y_train, y_train_pred)

print("Mean Squared Error (MSE) of Playlist with Polynomial Terms:", mse)
print("R² Score of Playlist with Polynomial Terms:", r2)

Mean Squared Error (MSE) of Playlist with Polynomial Terms: 5.413207999571759
R² Score of Playlist with Polynomial Terms: 0.023142942454236315


In [107]:
#Playlist linear regression with interaction terms

interaction = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
X_train_interact = interaction.fit_transform(X_train_scaled)

model = LinearRegression()
model.fit(X_train_interact, y_train)

y_train_pred = model.predict(X_train_interact)

mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

print("Mean Squared Error (MSE) Playlist with interaction terms:", mse_train)
print("R² Score Playlist with interaction terms:", r2_train)

Mean Squared Error (MSE) Playlist with interaction terms: 5.455864671615669
R² Score Playlist with interaction terms: 0.015445201827826716


In [108]:
#Playlist linear regression with multicollinearity

model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred_train = model.predict(X_train_scaled)

mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

print("MSE Playlist with multicollinearity:", mse_train)
print("R² Playlist with multicollinearity:", r2_train)

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)

coef_df = pd.DataFrame({
    'Feature': X_train_scaled_df.columns,
    'Coefficient': model.coef_
})

print("Coefficients:")
print(coef_df.sort_values(by='Coefficient', key=abs, ascending=False))


MSE Playlist with multicollinearity: 5.505702647235538
R² Playlist with multicollinearity: 0.006451537032031829
Coefficients:
           Feature  Coefficient
1           GameID    -0.555140
2          PlayKey     0.487088
5       PlayerGame     0.173335
12   PositionGroup     0.136501
4        PlayerDay    -0.106606
10  PlayerGamePlay     0.058692
9          Weather     0.036786
11        Position    -0.035816
0        PlayerKey    -0.033992
3   RosterPosition    -0.027886
6      StadiumType     0.027836
7        FieldType     0.007573
8      Temperature     0.005134


In [109]:
#Playlist linear regression with variance inflation factor and regression
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)

def calculate_vif(X_df):
    vif_data = pd.DataFrame()
    vif_data["feature"] = X_df.columns
    vif_data["VIF"] = [variance_inflation_factor(X_df.values, i) for i in range(X_df.shape[1])]
    return vif_data

vif_df = calculate_vif(X_train_scaled_df)
print("Initial VIF values:")
print(vif_df.sort_values(by="VIF", ascending=False))

high_vif_features = vif_df[vif_df["VIF"] > 10]["feature"].tolist()
X_filtered = X_train_scaled_df.drop(columns=high_vif_features)


model = LinearRegression()
model.fit(X_filtered, y_train)
y_pred_train = model.predict(X_filtered)

mse = mean_squared_error(y_train, y_pred_train)
r2 = r2_score(y_train, y_pred_train)

print("MSE of Playlist with VIF:", mse)
print("R² Playlist with VIF:", r2)

coef_df = pd.DataFrame({'Feature': X_filtered.columns, 'Coefficient': model.coef_})
print("\nCoefficients (Reduced Model):")
print(coef_df.sort_values(by='Coefficient', key=abs, ascending=False))

Initial VIF values:
           feature         VIF
1           GameID  755.349264
2          PlayKey  743.588397
0        PlayerKey    7.914023
4        PlayerDay    4.720910
5       PlayerGame    4.719784
11        Position    2.369268
3   RosterPosition    2.319514
12   PositionGroup    2.291310
8      Temperature    1.534066
6      StadiumType    1.390744
7        FieldType    1.093055
9          Weather    1.054010
10  PlayerGamePlay    1.014884
MSE of Playlist with VIF: 5.506586558175416
R² Playlist with VIF: 0.00629202817149499

Coefficients (Reduced Model):
           Feature  Coefficient
3       PlayerGame     0.172313
10   PositionGroup     0.133104
2        PlayerDay    -0.102533
0        PlayerKey    -0.098160
8   PlayerGamePlay     0.061235
7          Weather     0.037020
9         Position    -0.035891
4      StadiumType     0.029957
1   RosterPosition    -0.029075
5        FieldType     0.007005
6      Temperature     0.004241
